<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Exact Wald `GiRaFFEfood` Initial Data for `GiRaFFE`

## Author: Zach Etienne & Patrick Nelson
### Formatting improvements courtesy Brandon Clark

[comment]: <> (Abstract: TODO)

### NRPy+ Source Code for this module: [GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Exact_Wald.py](../../edit/in_progress/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Exact_Wald.py)

**Notebook Status:** <font color='green'><b> Validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation1). The initial data has validated against the original `GiRaFFE`, as documented [here](Tutorial-Start_to_Finish_UnitTest-GiRaFFEfood_NRPy.ipynb).

## Introduction: 
With the `GiRaFFE` evolution thorn constructed, we now need to "feed" our giraffe with initial data to evolve. There are several different choices of initial data we can use here; here, we will only be implementing the "Exact Wald" initial data, given by Table 3 in [the original paper](https://arxiv.org/pdf/1704.00599.pdf):
\begin{align}
A_{\phi} &= \frac{C_0}{2} r^2 \sin^2 \theta \\
E_{\phi} &= 2 M C_0 \left( 1+ \frac {2M}{r} \right)^{-1/2} \sin^2 \theta \\
\end{align}
(the unspecified components are set to 0). Here, $C_0$ is a constant that we will set to $1$ in our simulations. Now, to use this initial data scheme, we need to transform the above into the quantities actually tracked by `GiRaFFE` and HydroBase: $A_i$, $B^i$, $\tilde{S}_i$, $v^i$, and $\Phi$. Of these quantities, `GiRaFFEfood` will only set $A_i$, $v^i$, and $\Phi=0$, then call a separate function to calculate  $\tilde{S}_i$; `GiRaFFE` itself will call a function to set $B^i$ before the time-evolution begins. This can be done with eqs. 16 and 18, here given in that same order:
\begin{align}
v^i &= \alpha \frac{\epsilon^{ijk} E_j B_k}{B^2} -\beta^i \\
B^i &= \frac{[ijk]}{\sqrt{\gamma}} \partial_j A_k \\
\end{align}
In the simulations, $B^i$ will be calculated numerically from $A_i$; however, it will be useful to analytically calculate $B^i$ to use calculating the initial $v^i$.

This module requires the use of the NRPy+ [Shifted Kerr-Schild initial data module](../Tutorial-ADM_Initial_Data-ShiftedKerrSchild.ipynb)

<a id='toc'></a>

# Table of Contents:
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Import core NRPy+ modules and set NRPy+ parameters
1. [Step 2](#set_a_i): Set the vector $A_i$
1. [Step 3](#set_vi): Calculate $v^i$ from $A_i$ and $E_i$
1. [Step 4](#code_validation1): Code Validation against `GiRaFFEfood_NRPy.GiRaFFEfood_NRPy` NRPy+ Module
1. [Step 5](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Import core NRPy+ modules and set NRPy+ parameters \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Here, we will import the NRPy+ core modules, set the reference metric to Cartesian, and set commonly used NRPy+ parameters. We will also set up a parameter to determine what initial data is set up, although it won't do much yet.



In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 0.a: Import the NRPy+ core modules and set the reference metric to Cartesian
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import GiRaFFEfood_NRPy.GiRaFFEfood_NRPy_Common_Functions as gfcf # Some useful functions for GiRaFFE initial data.

import reference_metric as rfm
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

# Step 1a: Set commonly used parameters.
thismodule = "GiRaFFEfood_NRPy"

<a id='set_a_i'></a>

# Step 2: Set the vector $A_i$ \[Back to [top](#toc)\]
$$\label{set_a_i}$$

We will first build the fundamental vectors $A_i$ and $E_i$ in spherical coordinates (see [Table 3](https://arxiv.org/pdf/1704.00599.pdf)). Note that we use reference_metric.py to set $r$ and $\theta$ in terms of Cartesian coordinates; this will save us a step later when we convert to Cartesian coordinates. Since $C_0 = 1$,
$$A_{\phi} = \frac{1}{2} r^2 \sin^2 \theta.$$

In [2]:
# Step 2: Set the vectors A and E in Spherical coordinates
def Ar_EW(r,theta,phi, **params):
    return sp.sympify(0)

def Ath_EW(r,theta,phi, **params):
    return sp.sympify(0)

def Aph_EW(r,theta,phi, **params):
    # 1/2 r^2 \sin^2 \theta
    return (r * r * sp.sin(theta)**2)/2

<a id='set_vi'></a>

# Step 3: Calculate $v^i$ from $A_i$ and $E_i$ \[Back to [top](#toc)\]
$$\label{set_vi}$$

Next, we will code up the Valencia velocity, which will require us to first code the electric and magnetic fields. The magnetic field is simply $$B^i = \frac{[ijk]}{\sqrt{\gamma}} \partial_j A_k.$$

The electric field is
$$E_{\phi} = 2 M \left( 1+ \frac {2M}{r} \right)^{-1/2} \sin^2 \theta,$$\
Where the other components are $0$.

We can then calculate the the velocity as $$v_{(n)}^i = \frac{\epsilon^{ijk} E_j B_k}{B^2}.$$

In [3]:
#Step 3: Compute v^i from A_i and E_i
def ValenciavU_func_EW(**params):
    M = params["M"]
    gammaDD = params["gammaDD"] # Note that this must use a Cartesian basis!
    sqrtgammaDET = params["sqrtgammaDET"]
    KerrSchild_radial_shift = params["KerrSchild_radial_shift"]
    r     = rfm.xxSph[0] + KerrSchild_radial_shift # We are setting the data up in Shifted Kerr-Schild coordinates
    theta = rfm.xxSph[1]

    LeviCivitaTensorUUU = ixp.LeviCivitaTensorUUU_dim3_rank3(sqrtgammaDET)

    AD = gfcf.Axyz_func_spherical(Ar_EW,Ath_EW,Aph_EW,False,**params)
    # For the initial data, we can analytically take the derivatives of A_i
    ADdD = ixp.zerorank2()
    for i in range(3):
        for j in range(3):
            ADdD[i][j] = sp.simplify(sp.diff(AD[i],rfm.xx_to_Cart[j]))

    BU = ixp.zerorank1()
    for i in range(3):
        for j in range(3):
            for k in range(3):
                BU[i] += LeviCivitaTensorUUU[i][j][k] * ADdD[k][j]

    EsphD = ixp.zerorank1()
    # 2 M ( 1+ 2M/r )^{-1/2} \sin^2 \theta
    EsphD[2] = 2 * M * sp.sin(theta)**2 / sp.sqrt(1+2*M/r)

    ED = gfcf.change_basis_spherical_to_Cartesian_D(EsphD)

    return gfcf.compute_ValenciavU_from_ED_and_BU(ED, BU, gammaDD)

<a id='code_validation1'></a>

# Step 4: Code Validation against `GiRaFFEfood_NRPy.GiRaFFEfood_NRPy` NRPy+ module  \[Back to [top](#toc)\]
$$\label{code_validation1}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for the `GiRaFFE` Exact Wald initial data equations  we intend to use between
1. this tutorial and 
2. the NRPy+ [GiRaFFEfood_NRPy.GiRaFFEfood_NRPy_Exact_Wald](../edit/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Exact_Wald.py) module.


In [4]:
import GiRaFFEfood_NRPy.GiRaFFEfood_NRPy as gf
import BSSN.ShiftedKerrSchild as sks
sks.ShiftedKerrSchild(True)
import reference_metric as rfm   # NRPy+: Reference metric support
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

# Use the Jacobian matrix to transform the vectors to Cartesian coordinates.
drrefmetric__dx_0UDmatrix = sp.Matrix([[sp.diff(rfm.xxSph[0],rfm.xx[0]), sp.diff(rfm.xxSph[0],rfm.xx[1]), sp.diff(rfm.xxSph[0],rfm.xx[2])],
                                       [sp.diff(rfm.xxSph[1],rfm.xx[0]), sp.diff(rfm.xxSph[1],rfm.xx[1]), sp.diff(rfm.xxSph[1],rfm.xx[2])],
                                       [sp.diff(rfm.xxSph[2],rfm.xx[0]), sp.diff(rfm.xxSph[2],rfm.xx[1]), sp.diff(rfm.xxSph[2],rfm.xx[2])]])
dx__drrefmetric_0UDmatrix = drrefmetric__dx_0UDmatrix.inv()
gammaSphDD = ixp.zerorank2()
for i in range(3):
    for j in range(3):
        gammaSphDD[i][j] += sks.gammaSphDD[i][j].subs(sks.r,rfm.xxSph[0]).subs(sks.th,rfm.xxSph[1])

# gammaDD = ixp.zerorank2()
# for i in range(3):
#     for j in range(3):
#         for k in range(3):
#             for l in range(3):
#                 gammaDD[i][j] += drrefmetric__dx_0UDmatrix[(k,i)]*drrefmetric__dx_0UDmatrix[(l,j)]*gammaSphDD[k][l]
gammaDD = ixp.declarerank2("gammaDD","sym01")
unused_gammaUU,gammaDET = ixp.symm_matrix_inverter3x3(gammaDD)
sqrtgammaDET = sp.sqrt(gammaDET)

A_ewD = gfcf.Axyz_func_spherical(Ar_EW,Ath_EW,Aph_EW,stagger_enable = True,KerrSchild_radial_shift=sks.r0)
Valenciav_ewD = ValenciavU_func_EW(M=sks.M,KerrSchild_radial_shift=sks.r0,gammaDD=gammaDD,sqrtgammaDET=sqrtgammaDET)
# gf.GiRaFFEfood_NRPy_generate_initial_data(ID_type = "ExactWald", stagger_enable = True,M=sks.M,KerrSchild_radial_shift=sks.r0,gammaDD=gammaDD,sqrtgammaDET=sqrtgammaDET)

print("Consistency check between GiRaFFEfood_NRPy tutorial and NRPy+ module:")

def consistency_check(quantity1,quantity2,string):
    if quantity1-quantity2==0:
        print(string+" is in agreement!")
    else:
        print(string+" does not agree!")
        sys.exit(1)

for i in range(3):
    consistency_check(Valenciav_ewD[i],gf.ValenciavU[i],"ValenciavU"+str(i))
    consistency_check(A_ewD[i],gf.AD[i],"AD"+str(i))

gammaDD00*r0**2*xx0**2*xx2**2*(r0*sqrt(xx0**2 + xx1**2 + xx2**2) + xx0**2 + xx1**2 + xx2**2)**2/((xx0**2 + xx1**2 + xx2**2)*(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)*(xx0**4 + 2*xx0**2*xx1**2 + 2*xx0**2*xx2**2 + xx1**4 + 2*xx1**2*xx2**2 + xx2**4)**2) + 2*gammaDD01*r0**2*xx0*xx1*xx2**2*(r0*sqrt(xx0**2 + xx1**2 + xx2**2) + xx0**2 + xx1**2 + xx2**2)**2/((xx0**2 + xx1**2 + xx2**2)*(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)*(xx0**4 + 2*xx0**2*xx1**2 + 2*xx0**2*xx2**2 + xx1**4 + 2*xx1**2*xx2**2 + xx2**4)**2) + 2*gammaDD02*r0*xx0*xx2*((r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(2*xx0**2*xx2**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(7/2) - 2*xx0**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(9/2) + 2*xx0**2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**4 

AttributeError: module 'GiRaFFEfood_NRPy.GiRaFFEfood_NRPy' has no attribute 'ValenciavU'

In [5]:
old = sp.sympify("gammaDD00*r0**2*xx0**2*xx2**2*(r0*sqrt(xx0**2 + xx1**2 + xx2**2) + xx0**2 + xx1**2 + xx2**2)**2/((xx0**2 + xx1**2 + xx2**2)*(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)*(xx0**4 + 2*xx0**2*xx1**2 + 2*xx0**2*xx2**2 + xx1**4 + 2*xx1**2*xx2**2 + xx2**4)**2) + 2*gammaDD01*r0**2*xx0*xx1*xx2**2*(r0*sqrt(xx0**2 + xx1**2 + xx2**2) + xx0**2 + xx1**2 + xx2**2)**2/((xx0**2 + xx1**2 + xx2**2)*(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)*(xx0**4 + 2*xx0**2*xx1**2 + 2*xx0**2*xx2**2 + xx1**4 + 2*xx1**2*xx2**2 + xx2**4)**2) + 2*gammaDD02*r0*xx0*xx2*((r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(2*xx0**2*xx2**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(7/2) - 2*xx0**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(9/2) + 2*xx0**2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**4 + (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(9/2))/(2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(11/2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)) - (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(-2*xx1**2*xx2**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(7/2) + 2*xx1**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(9/2) - 2*xx1**2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**4 - (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(9/2))/(2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(11/2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)))*(r0*sqrt(xx0**2 + xx1**2 + xx2**2) + xx0**2 + xx1**2 + xx2**2)/(sqrt(xx0**2 + xx1**2 + xx2**2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)*(xx0**4 + 2*xx0**2*xx1**2 + 2*xx0**2*xx2**2 + xx1**4 + 2*xx1**2*xx2**2 + xx2**4)) + gammaDD11*r0**2*xx1**2*xx2**2*(r0*sqrt(xx0**2 + xx1**2 + xx2**2) + xx0**2 + xx1**2 + xx2**2)**2/((xx0**2 + xx1**2 + xx2**2)*(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)*(xx0**4 + 2*xx0**2*xx1**2 + 2*xx0**2*xx2**2 + xx1**4 + 2*xx1**2*xx2**2 + xx2**4)**2) + 2*gammaDD12*r0*xx1*xx2*((r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(2*xx0**2*xx2**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(7/2) - 2*xx0**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(9/2) + 2*xx0**2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**4 + (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(9/2))/(2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(11/2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)) - (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(-2*xx1**2*xx2**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(7/2) + 2*xx1**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(9/2) - 2*xx1**2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**4 - (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(9/2))/(2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(11/2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)))*(r0*sqrt(xx0**2 + xx1**2 + xx2**2) + xx0**2 + xx1**2 + xx2**2)/(sqrt(xx0**2 + xx1**2 + xx2**2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)*(xx0**4 + 2*xx0**2*xx1**2 + 2*xx0**2*xx2**2 + xx1**4 + 2*xx1**2*xx2**2 + xx2**4)) + gammaDD22*((r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(2*xx0**2*xx2**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(7/2) - 2*xx0**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(9/2) + 2*xx0**2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**4 + (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(9/2))/(2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(11/2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)) - (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(-2*xx1**2*xx2**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(7/2) + 2*xx1**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(9/2) - 2*xx1**2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**4 - (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(9/2))/(2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(11/2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)))**2")
new = sp.sympify("r0**2*gammaDD00*xx0**2*xx2**2*(r0*sqrt(xx0**2 + xx1**2 + xx2**2) + xx0**2 + xx1**2 + xx2**2)**2/((xx0**2 + xx1**2 + xx2**2)*(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)*(xx0**4 + 2*xx0**2*xx1**2 + 2*xx0**2*xx2**2 + xx1**4 + 2*xx1**2*xx2**2 + xx2**4)**2) + 2*r0**2*gammaDD01*xx0*xx1*xx2**2*(r0*sqrt(xx0**2 + xx1**2 + xx2**2) + xx0**2 + xx1**2 + xx2**2)**2/((xx0**2 + xx1**2 + xx2**2)*(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)*(xx0**4 + 2*xx0**2*xx1**2 + 2*xx0**2*xx2**2 + xx1**4 + 2*xx1**2*xx2**2 + xx2**4)**2) + r0**2*gammaDD11*xx1**2*xx2**2*(r0*sqrt(xx0**2 + xx1**2 + xx2**2) + xx0**2 + xx1**2 + xx2**2)**2/((xx0**2 + xx1**2 + xx2**2)*(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)*(xx0**4 + 2*xx0**2*xx1**2 + 2*xx0**2*xx2**2 + xx1**4 + 2*xx1**2*xx2**2 + xx2**4)**2) + 2*r0*gammaDD02*xx0*xx2*((r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(2*xx0**2*xx2**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(7/2) - 2*xx0**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(9/2) + 2*xx0**2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**4 + (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(9/2))/(2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(11/2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)) - (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(-2*xx1**2*xx2**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(7/2) + 2*xx1**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(9/2) - 2*xx1**2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**4 - (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(9/2))/(2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(11/2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)))*(r0*sqrt(xx0**2 + xx1**2 + xx2**2) + xx0**2 + xx1**2 + xx2**2)/(sqrt(xx0**2 + xx1**2 + xx2**2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)*(xx0**4 + 2*xx0**2*xx1**2 + 2*xx0**2*xx2**2 + xx1**4 + 2*xx1**2*xx2**2 + xx2**4)) + 2*r0*gammaDD12*xx1*xx2*((r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(2*xx0**2*xx2**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(7/2) - 2*xx0**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(9/2) + 2*xx0**2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**4 + (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(9/2))/(2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(11/2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)) - (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(-2*xx1**2*xx2**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(7/2) + 2*xx1**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(9/2) - 2*xx1**2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**4 - (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(9/2))/(2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(11/2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)))*(r0*sqrt(xx0**2 + xx1**2 + xx2**2) + xx0**2 + xx1**2 + xx2**2)/(sqrt(xx0**2 + xx1**2 + xx2**2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)*(xx0**4 + 2*xx0**2*xx1**2 + 2*xx0**2*xx2**2 + xx1**4 + 2*xx1**2*xx2**2 + xx2**4)) + gammaDD22*((r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(2*xx0**2*xx2**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(7/2) - 2*xx0**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(9/2) + 2*xx0**2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**4 + (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(9/2))/(2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(11/2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)) - (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(-2*xx1**2*xx2**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(7/2) + 2*xx1**2*(r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2 + xx2**2)**(9/2) - 2*xx1**2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**4 - (r0 + sqrt(xx0**2 + xx1**2 + xx2**2))*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(9/2))/(2*(xx0**2 + xx1**2)*(xx0**2 + xx1**2 + xx2**2)**(11/2)*sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)))**2")
print(sp.simplify(old-new))

0


<a id='latex_pdf_output'></a>

# Step 5: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFEfood_NRPy.pdf](Tutorial-GiRaFFEfood_NRPy.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [ ]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFEfood_NRPy_Exact_Wald",location_of_template_file=os.path.join(".."))